Tests the CheapFake module in cheapfake.contrib.models.CheapFake, in particular the loading of LipNet and FAN.

In [1]:
import os
import time
import warnings

import cheapfake.contrib.dataset as dataset
from cheapfake.contrib.models import CheapFake

In [2]:
cheapfake_model = CheapFake()

In [3]:
random_seed = 41
root_path = "/Users/shu/Documents/Datasets/DFDC_sample/test"

dfdataset = dataset.DeepFakeDataset(root_path=root_path, return_tensor=True, random_seed=random_seed, sequential_frames=True, sequential_audio=True, stochastic=True)
frames, audio, audio_stft = 

400
